## Creating csv file

In [52]:
import codecs
from bs4 import BeautifulSoup
import csv
import os

In [62]:
csv_file.close()

In [63]:
csv_file = open('dou_vacancies.csv', 'w', encoding='utf-8')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['headline', 'description'])

22

In [64]:
directory = 'dou/rezult'
for filename in os.listdir(directory):    
    file = codecs.open(directory + '/' + filename, "r", 'utf-8')
    soup = BeautifulSoup(file)
    headline = soup.find('h1', class_="g-h2").get_text()
    vacancy = soup.find('div', class_="l-vacancy")
    descriptions = vacancy.find_all('div', class_="text b-typo vacancy-section")
    text = ''
    for description in descriptions:
        text += description.get_text()
    csv_writer.writerow([headline, text])

In [ ]:
import pandas as pd

In [65]:
df = pd.read_csv('dou_vacancies.csv')

In [67]:
df.head()

,headline,description
0,"Junior ML engineer (Computer Vision, 3d)",\r\n— Python— хорошая математическая база для ...
1,Senior Java/BigData Engineer,\r\nExperience in Java development (5 years+);...
2,Java Engineer for HighLoad/BigData project | SM,\r\n— Professional Java experience 3+ years— O...
3,NLP Engineer,"\r\n— 2+ years experience with Python, includi..."
4,Data Engineer,\r\n— 2+ years of hands-on experience as a Dat...


## Text processing

In [103]:
import re
import nltk
from nltk.tokenize import word_tokenize
#nltk.download('stopwords')
#nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from string import punctuation
punctuation += '—'

In [90]:
stop = set(stopwords.words('english'))
punct = list(punctuation)

In [83]:
def clean_text(text):
    text = text.replace(u'\xa0', u' ')
    text_eng = re.sub('[^a-zA-Z0-9-+ ]+', '', text)
    return text_eng

In [92]:
def remove_stop_words(text):
    lst = word_tokenize(text)
    lst = [word for word in lst if word not in stop and word not in punct]
    return lst

In [123]:
def leave_nouns(lst):
    new_lst = [word for (word,pos) in nltk.pos_tag(lst) if pos[:2] == 'NN']
    return new_lst

In [99]:
df['description'] = df.description.apply(clean_text)

In [113]:
df['description'] = df.description.apply(remove_stop_words)

In [125]:
df['description'] = df.description.apply(leave_nouns)

In [257]:
df['is_data_engineer'] = df['headline'].apply(lambda x: 'data' in x.lower())

In [258]:
df.head()

,headline,description,is_data_engineer
0,"Junior ML engineer (Computer Vision, 3d)","[Python, machine, learning, Numpy, Pandas, Sci...",False
1,Senior Java/BigData Engineer,"[Experience, Java, development, years+Experien...",True
2,Java Engineer for HighLoad/BigData project | SM,"[Java, experience, years, OOPdesign, proficien...",True
3,NLP Engineer,"[years, Python, engineering, experience, API, ...",False
4,Data Engineer,"[years, experience, Data, Engineer, experience...",True


In [259]:
df['is_data_engineer'].value_counts()

True     118
False     38
Name: is_data_engineer, dtype: int64

In [308]:
df_engineers = df.query('is_data_engineer == True').copy()

In [311]:
def leave_relevant_technologies(lst_input):
    techno_list = ['Airflow', 'Apache', 'Asure', 'Bash', 'BigQuery', 'Databricks', 'Data Bricks', 'Data Factory', 
            'Delta Lake', 'Docker', 'ETL', 'Extract Load Transform', 'Fiddler', 'Flink', 'GCP', 'Google Cloud', 
            'Jenkins', 'Jira', 'Impala', 'InfluxDB',  'Kinesis', 'Linux', 'Mesos',  'Postman', 'PowerBI', 'PySpark', 
                   'R-Studio',   'snowflake', 'Spring',
            'SSIS', 'SSAS', 'QA', 'Tableau', 'Tensorflow']
    message_brokers = ['Kafka', 'RabbitMQ', 'AMQP', 'Redis']
    databases = ['SQL', 'NoSQL', 'SQL Server', 'Cassandra', 'Cosmos DB', 'MongoDB', 
                 'MySQL', 'RabGTD', 'Rabbit', 'Oracle', 'PostgreSQL']
    big_data = ['Hadoop', 'Hive', 'Hbase', 'MapReduce', 'Map Reduce', 'Pig', 'Spark']
    amazon = ['AWS', 'Redshift']
    formats = ['JSON', 'XML']
    prog_lang = ['C', 'C++', 'R', 'Kotlin', 'Scala', 'Java', 'JS', 'JavaScript', 'Python', 'ReactJS']
    soft_skills = ['Scrum', 'Agile']
    final_list = techno_list + message_brokers + databases + big_data + prog_lang + soft_skills
    lst_input = [w.lower() for w in lst_input]
    lst = [word for word in final_list if word.lower() in lst_input]
    return list(set(lst))

In [312]:
df_engineers['description'] = df_engineers['description'].apply(leave_relevant_technologies)
df_engineers.drop(['is_data_engineer'], axis=1, inplace=True)

In [313]:
df_engineers.head(10)

,headline,description
1,Senior Java/BigData Engineer,"[Spark, Jenkins, SQL, Linux, Java, Apache]"
2,Java Engineer for HighLoad/BigData project | SM,"[MongoDB, Linux, Java]"
4,Data Engineer,"[BigQuery, Kinesis, Airflow, Python, ETL]"
5,Middle Data Engineer (#14074599),"[Python, snowflake, SQL]"
6,Data engineer,"[Kafka, Spark, Python, Agile, Scala, Apache]"
7,Middle Data Warehouse Engineer,"[SQL, MySQL, ETL, Tableau, PostgreSQL]"
8,Senior Scala/Java Engineer for Exabeam (Data L...,"[Kafka, Python, Bash, MongoDB, Linux, Java, Sc..."
9,Middle Data Engineer,"[SSAS, SSIS]"
10,Data Engineer (Scala/Spark),"[Java, ETL, Scala, NoSQL, PostgreSQL, Kafka, J..."
13,Lead Big Data Engineer,"[Python, Bash, SQL, PySpark, ETL, Hadoop]"


In [314]:
result = df_engineers['description'].tolist()

In [315]:
result = [item for sublist in result for item in sublist]

In [316]:
len(result)

723

In [317]:
from collections import Counter
cnt = Counter(result)

In [318]:
cnt.most_common()

[('Python', 74),
 ('SQL', 73),
 ('Spark', 50),
 ('Kafka', 41),
 ('ETL', 37),
 ('Java', 31),
 ('Airflow', 28),
 ('PostgreSQL', 26),
 ('Hadoop', 25),
 ('Apache', 23),
 ('Scala', 22),
 ('Docker', 20),
 ('MySQL', 18),
 ('NoSQL', 18),
 ('Agile', 15),
 ('GCP', 14),
 ('Hive', 14),
 ('MongoDB', 13),
 ('BigQuery', 13),
 ('Linux', 10),
 ('Jenkins', 9),
 ('Cassandra', 9),
 ('QA', 9),
 ('Tableau', 8),
 ('PySpark', 8),
 ('MapReduce', 8),
 ('Redis', 7),
 ('Oracle', 7),
 ('Hbase', 7),
 ('Kinesis', 6),
 ('snowflake', 6),
 ('Jira', 6),
 ('Scrum', 6),
 ('Bash', 5),
 ('RabbitMQ', 5),
 ('SSIS', 4),
 ('JavaScript', 4),
 ('C', 4),
 ('R', 4),
 ('Pig', 4),
 ('PowerBI', 4),
 ('Databricks', 4),
 ('Impala', 3),
 ('Spring', 3),
 ('Tensorflow', 3),
 ('C++', 2),
 ('Fiddler', 2),
 ('Kotlin', 2),
 ('Postman', 2),
 ('SSAS', 1),
 ('Flink', 1),
 ('ReactJS', 1),
 ('AMQP', 1),
 ('Mesos', 1),
 ('R-Studio', 1),
 ('Rabbit', 1)]

## How I made a list of relevant techonologies

In [141]:
df_engineers['text'] = df_engineers['description'].apply(lambda x : " ".join(x))

In [144]:
all_text = df_engineers['text'].str.cat(sep=', ')

In [149]:
import spacy
from spacy.lang.en import English
nlp = spacy.load('en_core_web_md')

In [150]:
doc = nlp(all_text)

In [162]:
list_technologies = []

In [163]:
for ent in doc.ents:
    if ent.label_ != 'DATE' and ent.label_ != 'GPE':        
        list_technologies.append(ent.text)

In [166]:
set_of_technologies = set(list_technologies)

In [228]:
#set_of_technologies

In [167]:
len(set_of_technologies)

752

In [196]:
for ent in doc.ents:
    #if ent.label_ == 'CARDINAL':
    if ent.label_ != 'DATE' and ent.label_ != 'GPE' and ent.label_ != 'LOC' and ent.label_!= 'CARDINAL' and ent.label_ != 'TIME': 
        print(ent.text, ent.label_)

HadoopTechnical ORG
Jenkins CICD PERSON
Intellias PERSON
Personal Development PlanRegular ORG
Lviv KyivFor PERSON
Lviv KyivSolve ORG
Big Data ORG
SQL ORG
standardsUse ORG
SQLNoSQL PRODUCT
Familiarity Adtech ORG
Familiarity Aerospike ORG
Xenoss PERSON
Data Engineer ORG
Machine Learning Data Science Familiar ML Experience ORG
S3 ORG
ETL ORG
BI ORG
Looker ORG
AWS QuickSight Google Data ORG
Fridge ORG
SQL ORG
dbtWe ORG
Healthcare ORG
Apache Spark Kafka PRODUCT
SQL ORG
SQL BigData ORG
English NORP
Experience CICD Knowledge ORG
Vystavkovyi PRODUCT
UI ORG
Data WarehouseData Integration Knowledge ORG
Kimball Inmon Hands- ORG
Business IntelligenceAnalytical ORG
Tableau Extensive ORG
SQL ORG
DBMS ORG
Amazon Redshift ORG
GPC ORG
WFH ORG
AWS ORG
ETL ORG
Scala Java PERSON
Elasticsearch Hadoop MongoDB ORG
Kafka Zookeeper Docker ORG
Linux Bash PERSON
Ansible Kubernetes PERSON
CICD ORG
PaaSSaaS ORG
US Dedicated HR Client ORG
Big Data Infrastructure ORG
second ORDINAL
Scala ORG
Data Engineer ORG
Busine

In [222]:
techno_list = ['Airflow', 'AWS', 'Apache', 'Asure', 'Bash', 'BigQuery', 'Cassandra', \
              'Databricks', 'Delta Lake', 'Docker', 'ETL', 'Extract Load Transform', 'Fiddler', 'Flink', 'GCP', 'Jenkins', 'Jira', 'JSON',\
              'Hadoop', 'Kafka', 'Kinesis', 'Linux', 'Mesos', 'MongoDB', 'MySQL', 'Oracle', 'NoSQL',\
              'Postman', 'PowerBI', 'PySpark', 'QA', 'R-Studio', 'Rabbit', 'ReactJS',\
              'SQL', 'Tableau', 'Tensorflow', 'XML']

In [229]:
prog_lang = ['C', 'C++', 'Kotlin', 'Scala' 'Java', 'JS', 'JavaScript', 'Python']

In [230]:
soft_skills = ['Scrum', 'agile']
final_list = techno_list + prog_lang + soft_skills